# Automatic Speech Recognition with Speaker Diarization

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Install TorchAudio
!pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html

# Introduction

Speaker diarization lets us figure out "who spoke when" in the transcription. Without speaker diarization, we cannot distinguish the speakers in the transcript generated from automatic speech recognition (ASR). Nowadays, ASR combined with speaker diarization has shown immense use in many tasks, ranging from analyzing meeting transcription to media indexing. 

In this tutorial, we demonstrate how we can get ASR transcriptions combined with speaker labels. Since we don't include detailed process of getting ASR result or diarization result, please refer to the following links for more in-depth description.

If you need detailed understanding of transcribing words with ASR, refer to this [ASR Tutorial](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/asr/ASR_with_NeMo.ipynb) tutorial.


For detailed parameter setting and execution of speaker diarization, refer to this [Diarization Inference](https://github.com/NVIDIA/NeMo/blob/stable/tutorials/speaker_tasks/Speaker_Diarization_Inference.ipynb) tutorial.


An example script that runs ASR and speaker diarization together can be found at [ASR with Diarization](https://github.com/NVIDIA/NeMo/blob/main/examples/speaker_tasks/diarization/asr_with_diarization.py).

### Speaker diarization in ASR pipeline

Speaker diarization result in ASR pipeline should align well with ASR output. Thus, we use ASR output to create Voice Activity Detection (VAD) timestamps to obtain segments we want to diarize. The segments we obtain from the VAD timestamps are further segmented into sub-segments in the speaker diarization step. Finally, after obtaining the speaker labels from speaker diarization, we match the decoded words with speaker labels to generate a transcript with speaker labels.

    ASR → VAD timestamps and decoded words → speaker diarization → speaker label matching

### Import libraries

Let's first import nemo asr and other libraries for visualization purposes.

In [ ]:
import nemo.collections.asr as nemo_asr
import numpy as np
from IPython.display import Audio, display
import librosa
import os
import wget
import matplotlib.pyplot as plt

import nemo
from nemo.collections.asr.parts.utils.diarization_utils import ASR_DIAR_OFFLINE

import glob

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
def read_file(path_to_file):
    with open(path_to_file) as f:
        contents = f.read().splitlines()
    return contents

We demonstrate this tutorial using a merged an4 audio, that has two speakers (male and female) speaking dates in different formats. If the audio does not exist, we download it, and listen to it

In [ ]:
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'data')
os.makedirs(data_dir, exist_ok=True)
an4_audio_url = "https://nemo-public.s3.us-east-2.amazonaws.com/an4_diarize_test.wav"
if not os.path.exists(os.path.join(data_dir,'an4_diarize_test.wav')):
    AUDIO_FILENAME = wget.download(an4_audio_url, data_dir)
else:
    AUDIO_FILENAME = os.path.join(data_dir,'an4_diarize_test.wav')

audio_file_list = glob.glob(f"{data_dir}/*.wav")
print("Input audio file list: \n", audio_file_list)

signal, sample_rate = librosa.load(AUDIO_FILENAME, sr=None)
display(Audio(signal,rate=sample_rate))

`display_waveform()` and `get_color()` functions are defined for displaying the waveform with diarization results.

In [ ]:
def display_waveform(signal,text='Audio',overlay_color=[]):
    fig,ax = plt.subplots(1,1)
    fig.set_figwidth(20)
    fig.set_figheight(2)
    plt.scatter(np.arange(len(signal)),signal,s=1,marker='o',c='k')
    if len(overlay_color):
        plt.scatter(np.arange(len(signal)),signal,s=1,marker='o',c=overlay_color)
    fig.suptitle(text, fontsize=16)
    plt.xlabel('time (secs)', fontsize=18)
    plt.ylabel('signal strength', fontsize=14);
    plt.axis([0,len(signal),-0.5,+0.5])
    time_axis,_ = plt.xticks();
    plt.xticks(time_axis[:-1],time_axis[:-1]/sample_rate);
    
COLORS="b g c m y".split()

def get_color(signal,speech_labels,sample_rate=16000):
    c=np.array(['k']*len(signal))
    for time_stamp in speech_labels:
        start,end,label=time_stamp.split()
        start,end = int(float(start)*16000),int(float(end)*16000),
        if label == "speech":
            code = 'red'
        else:
            code = COLORS[int(label.split('_')[-1])]
        c[start:end]=code
    
    return c 

Using the above function, we can display the waveform of the example audio clip. 

In [ ]:
display_waveform(signal)

### Parameter setting for ASR and diarization
First, we need to setup the following parameters for ASR and diarization. We start our demonstration by first transcribing the audio recording using our pretrained ASR model `QuartzNet15x5Base-En` and use the CTC output probabilities to get timestamps for the spoken words. We then use these timestamps to get speaker label information using speaker diarizer model. 

In [ ]:
CONFIG_URL = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/speaker_diarization.yaml"

params = {
    "time_stride": 0.02,  # This should not be changed if you are using QuartzNet15x5Base.
    "offset": -0.18,  # This should not be changed if you are using QuartzNet15x5Base.
    "round_float": 2,
    "window_length_in_sec": 1.5,
    "shift_length_in_sec": 0.25,
    "print_transcript": False,
    "threshold": 50,  # minimun width to consider non-speech activity
    "external_oracle_vad": False,
    "diar_config_url": CONFIG_URL,
    "ASR_model_name": 'QuartzNet15x5Base-En',
}

Let's create an instance from ASR_DIAR_OFFLINE class. We pass the ``params`` variable to setup the parameters for both ASR and diarization. 

In [ ]:
asr_diar_offline = ASR_DIAR_OFFLINE(params)
asr_model = asr_diar_offline.set_asr_model(params['ASR_model_name'])

We will create folders that we need for storing VAD stamps and ASR/diarization results.

Under the folder named ``asr_with_diar``, the following folders will be created.

- ``oracle_vad``
- ``json_result``
- ``transcript_with_speaker_labels``


In [ ]:
asr_diar_offline.create_directories()

print("Folders are created as below.")
print("VAD file path: \n", asr_diar_offline.oracle_vad_dir)
print("JSON result path: \n", asr_diar_offline.json_result_dir)
print("Transcript result path: \n", asr_diar_offline.trans_with_spks_dir)

### Run ASR and get word timestamps
Before we run speaker diarization, we should run ASR and get the ASR output to generate VAD timestamps and decoded words. `run_ASR()` function extracts word sequence, logit values for each frame and timestamps for each token (character). These three types of results are included in ``ASR_output`` variable. 

In [ ]:
ASR_output = asr_diar_offline.run_ASR(asr_model, audio_file_list)

print("Decoded word output: \n", ASR_output[0][0])
print("Logit values for each frame:  \n",ASR_output[0][1].shape, ASR_output[0][1])
print("Framelevel timestamps for each token: \n", ASR_output[0][2])

The following three variables are obtained from `get_speech_labels_list()` function.

- words List[str]: contains the sequence of words.
- spaces List[int]: contains frame level index of the end of the last word and the start time of the next word. 
- word_ts List[int]: contains frame level index of the start and the end of each word.

In [ ]:
words, spaces, word_ts = asr_diar_offline.get_speech_labels_list(ASR_output, audio_file_list)

print("Transcribed words: \n", words[0])
print("Spaces between words: \n", spaces[0])
print("Timestamps for the words: \n", word_ts[0])


Then we multiply `params['time_stride']=0.02` to get timestamp in second.

### Run diarization with extracted VAD timestamp


We need to convert ASR based VAD output (*.rttm format) to VAD manifest (*.json format) file. The following function converts the rttm files into manifest file and returns the path for manifest file.

In [ ]:
vad_manifest_path = asr_diar_offline.write_VAD_rttm(asr_diar_offline.oracle_vad_dir, audio_file_list)

print("VAD manifest file path: \n", vad_manifest_path)
print("VAD Manifest file content:")
pp.pprint(read_file(vad_manifest_path))

Now that all the components for diarization is ready, let's run diarization by calling `run_diarization()` function.

In [ ]:
num_speakers = None # If we know the number of speakers, we can assign "2".
pretrained_speaker_model = 'ecapa_tdnn'

asr_diar_offline.run_diarization(audio_file_list, vad_manifest_path, num_speakers, pretrained_speaker_model)

print(nemo.__file__)

`run_diarization()` function will create `./asr_with_diar/oracle_vad/pred_rttm/an4_diarize_test.rttm` file. Let's see what is written in this `rttm` file.

In [ ]:
predicted_speaker_label_rttm_path = f"{ROOT}/asr_with_diar/oracle_vad/pred_rttms/an4_diarize_test.rttm"
pred_rttm = read_file(predicted_speaker_label_rttm_path)

pp.pprint(pred_rttm)

Let's check out the diarization output. `get_diarization_labels()` function extracts the estimated speaker label information with timestamps from the predicted rttm file.

In [ ]:
diar_labels = asr_diar_offline.get_diarization_labels(audio_file_list)

print("Diarization Labels:")
pp.pprint(diar_labels[0])

color = get_color(signal, diar_labels[0])
display_waveform(signal,'Audio with Speaker Labels',color)
display(Audio(signal,rate=16000))

### Check the transcription output
Now we've done all the processes for running ASR and diarization, let's match the diarization result with ASR result and get the final output. `write_json_and_transcript()` function matches diarization output `diar_labels` with `words` using the timestamp information `word_ts`. 

In [ ]:
asr_output_dict = asr_diar_offline.write_json_and_transcript(audio_file_list, diar_labels, words, word_ts)

After running `write_json_and_transcript()` function, the transcription output will be located in `./asr_with_diar/transcript_with_speaker_labels` folder, which shows **start time to end time of the utterance, speaker ID, and words spoken** during the notified time.

In [ ]:
transcription_path_to_file = f"{ROOT}/asr_with_diar/transcript_with_speaker_labels/an4_diarize_test.txt"
transcript = read_file(transcription_path_to_file)

pp.pprint(transcript)

Another output is transcription output in JSON format, which is saved in `./asr_with_diar/json_result`. 

In the JSON format output, we include information such as **transcription, estimated number of speakers (variable named `speaker_count`), start and end time of each word and most importantly, speaker label for each word.**

In [ ]:
transcription_path_to_file = f"{ROOT}/asr_with_diar/json_result/an4_diarize_test.json"
json_contents = read_file(transcription_path_to_file)

pp.pprint(json_contents)